In [ ]:
"""
note
2025.03.24 reset
"""

'\nnote\n2025.03.13\n미니 프로젝트 시작.\n상관계수를 직접 구해보는 과정을 해보고 싶었으나 아직 통계와 분석에 대한 이해가 떨어져 바로 결정트리 구현을 진행함.\n2025.03.14\n넘파이 없이 구현을 목표로 했으나 특징벡터 4개, 클래스 3개로 계산량이 너무 커서 넘파이를 사용해 구현,\n추후 상관계수 계산 후 특징벡터 2개를 이용한 이진, 혹은 3 클래스 분류를 진행하고자함.\n'

In [1]:
import numpy as np
import pandas as pd

np.random.seed(0)

In [2]:
df = pd.read_csv('Iris.csv', index_col=0)
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
Id,,,,,
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
df.columns = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'target']
df['target'] = df['target'].map({'Iris-versicolor':0, 'Iris-setosa':1, 'Iris-virginica':2})

In [4]:
df

,SepalLength,SepalWidth,PetalLength,PetalWidth,target
Id,,,,,
1,5.1,3.5,1.4,0.2,1
2,4.9,3.0,1.4,0.2,1
3,4.7,3.2,1.3,0.2,1
4,4.6,3.1,1.5,0.2,1
5,5.0,3.6,1.4,0.2,1
...,...,...,...,...,...
146,6.7,3.0,5.2,2.3,2
147,6.3,2.5,5.0,1.9,2
148,6.5,3.0,5.2,2.0,2
